In [9]:
import ollama
import json
import pandas as pd

## Define Schema

In [2]:
schema = """
Dataset fields:

MODES:
- bus
- tube
- national-rail

ACCESSIBILITY:
- Not Accessible
- Partially Accessible
- Partially Accessible - Interchange Only
- Fully Accessible

NIGHT_TUBE:
- Yes
- No
"""

## Function to convert text -> WHERE clause

In [68]:
def convert_to_where(user_input):
    prompt = f"""
You are generating a pandas DataFrame.query() expression.

Return ONLY the expression.
Do NOT include:
- DataFrame.query(
- df.query(
- WHERE
- explanations
- markdown
- surrounding quotes

Rules:
- Use == for equality
- Use and / or in lowercase
- Include spaces around AND / OR
- Field names are case-sensitive and must match exactly

Valid fields:
- MODES
- ACCESSIBILITY
- NIGHT_TUBE

{schema}

User request:
{user_input}

WHERE clause:
"""
    
    response = ollama.chat(
        model = 'llama3',
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return response['message']['content'].strip()

## Interactive loop

In [42]:
if __name__ == "__main__":
    while True:
        user_text = input("\nEnter query (or 'quit'): ")
        if user_text.lower() == "quit":
            break

        where_clause = convert_to_where(user_text)
        print("\nGenerated WHERE clause:")
        print(where_clause)


Generated WHERE clause:
Accessibility == 'Fully Accessible'


## Test Filtering with dataset

In [69]:
df = pd.read_csv("data/Underground_Stations/Underground_Stations.csv")

In [ ]:
user_input = "show me fully accessible stations with night tube"

where_clause = convert_to_where(user_input)

print("Generated expression: ", where_clause)

filtered = df.query(where_clause)

filtered.head()


Generated expression:  MODES == 'tube' and ACCESSIBILITY == 'Fully Accessible' and NIGHT_TUBE == 'Yes'


,X,Y,OBJECTID,NAME,LINES,ATCOCODE,MODES,ACCESSIBILITY,NIGHT_TUBE,NETWORK,DATASET_LAST_UPDATED,FULL_NAME
56,530174.7976,182874.7747,167,King's Cross St. Pancras,"Metropolitan, Northern, Piccadilly, Circle, Vi...",940GZZLUKSX,tube,Fully Accessible,Yes,London Underground,2021/11/29 00:00:00+00,King's Cross St. Pancras station
75,531052.3817,175428.6555,186,Brixton,Victoria,940GZZLUBXN,tube,Fully Accessible,Yes,London Underground,2021/11/29 00:00:00+00,Brixton station
102,535456.6419,179467.0078,213,Canada Water,Jubilee,940GZZLUCWR,tube,Fully Accessible,Yes,London Underground,2021/11/29 00:00:00+00,Canada Water station
104,537677.3398,180139.7318,215,Canary Wharf,Jubilee,940GZZLUCYF,tube,Fully Accessible,Yes,London Underground,2021/11/29 00:00:00+00,Canary Wharf station
105,539215.8219,179859.3744,216,North Greenwich,Jubilee,940GZZLUNGW,tube,Fully Accessible,Yes,London Underground,2021/11/29 00:00:00+00,North Greenwich station
